In [1]:
from collections import namedtuple
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

# フレームの日射熱取得率を計算するモジュール

## 1. Functions  

フレームの熱貫流率、屋外側表面日射吸収率、屋外側展開面積比からフレームの日射熱取得率を計算する。

### 入力値  

$\alpha_{fr,ex}$ ：フレームの屋外側表面日射吸収率  
$ U_{fr} $ ：フレームの熱貫流率 [W/(m<sup>2</sup>)･K)]   
$ F_{de,ex} $ ：フレームの屋外側展開面積比   
$ h_{fr,ex} $ ：フレームの屋外側表面熱伝達率 [W/(m<sup>2</sup>)･K)]   

### 出力値  

$ \eta_{fr} $ ：フレームの日射熱取得率  

### 計算方法  

#### データ構造と値の範囲  

im・$ \alpha_{fr,ex} $(または$ \alpha_{fr,ex} $の表ID)・$ U_{fr} $(または$ U_{fr} $の表ID)・$ F_{de,ex} $(または$ F_{de,ex} $の表ID)・$ h_{fr,ex} $(または$ h_{fr,ex} $の表ID) の組み合わせをクラス『ghf＿input』と定義する。  

In [2]:
class ghf_input():
    def __init__( self, im, ap, Up, Fp, hp ):
        self.im = im
        self.ap = ap
        self.Up = Up
        self.Fp = Fp
        self.hp = hp
        
#im：データ取得方法（0：表参照入力、1：直接入力）
#ap：imが0の場合は、屋外側表面日射吸収率のID（0：省エネ基準住宅、1：省エネ基準非住宅、2～10：JIS A 2103の材質・色別）
#    imが1の場合は、屋外側表面日射吸収率
#Up：imが0の場合は、熱貫流率のID（0：JIS A 2103の樹脂、1：JIS A 2103のアルミ樹脂複合、2：JIS A 2103のアルミ）
#    imが1の場合は、熱貫流率[W/(m2･K)]
#Fp：imが0の場合は、屋外側展開面積比のID（0：省エネ基準住宅、1：省エネ基準非住宅、2～5：JIS A 2103の開閉形式別）
#    imが1の場合は、屋外側展開面積比
#hp：imが0の場合は、屋外側表面熱伝達率のID（0：省エネ基準住宅、1：省エネ基準非住宅、2：JIS A 2103の夏期、3：JIS A 2103の冬期）
#    imが1の場合は、屋外側表面熱伝達率[W/(m2･K)]

#### フレームの日射熱取得率の計算方法  

フレームの日射熱取得率$\eta_{fr}$の計算方法を以下に示す。  

（参考：JIS A 2103：2014 5.3 フレームの日射熱取得率）

$$
\begin{eqnarray}
 &\displaystyle \eta_{fr} = \frac {\alpha_{fr,ex} \cdot U_{fr}}{ F_{de,ex} \cdot h_{fr,ex}}& \qquad\qquad\text{(1)} \nonumber\\
\end{eqnarray}
$$

ここで、  
$ \alpha_{fr,ex} $ ：フレームの屋外側表面日射吸収率  
$ U_{fr} $ ：フレームの熱貫流率 [W/(m<sup>2</sup>･K)]  
$ F_{de,ex} $ ：フレームの屋外側展開面積比  
$ h_{fr,ex} $ ：フレームの屋外側表面熱伝達率 [W/(m<sup>2</sup>･K)]  

$$
\begin{eqnarray}
 &\displaystyle F_{de,ex} = \frac {A_{de,ex}}{A_{fr,in}}& \qquad\qquad\text{(2)} \nonumber\\
\end{eqnarray}
$$

ここで、  
$ A_{de,ex} $ ：屋外側フレーム展開面積 (m<sup>2</sup>)  
$ A_{fr,in} $ ：室内側投影フレーム面積 (m<sup>2</sup>)  

である。 

$ \alpha_{fr,ex} $の代表的な値を以下に示す（参考：平成25年省エネルギー基準に準拠した算定・判断の方法及び解説 II 住宅 第四部第三節及びJIS A 2103：2014 表A.1）。

| ID   | 規格・基準         | 材質         | 色           | 屋外側表面日射吸収率$\alpha_{fr,ex}$ |
| ---: | :---               | :---         | :---         | ---:                                 |
| 0    | 省エネ基準(住宅)   |―            | ―           | 0.80                                 |
| 1    | 省エネ基準(非住宅) |―            | ―           | 0.80                                 |
| 2    | JIS A 2103         | 樹脂         | ブラック     | 0.69                                 |
| 3    |                    |              | ブラウン     | 0.68                                 |
| 4    |                    |              | グレイ       | 0.67                                 |
| 5    |                    |              | ホワイト     | 0.22                                 |
| 6    |                    | アルミニウム | ブラック     | 0.87                                 |
| 7    |                    |              | ブラウン     | 0.76                                 |
| 8    |                    |              | グレイ       | 0.47                                 |
| 9    |                    |              | シルバー     | 0.26                                 |
| 10   |                    |              | ホワイト     | 0.24                                 |

$ U_{fr} $の代表的な値を以下に示す（参考：JIS A 2103：2014 表A.3）。  

| ID   | 規格・基準         | 材質           | 熱貫流率$U_{fr}$ [W/(m<sup>2</sup>･K)] |
| ---: | :---               | :---           | ---:                                   |
| 0    | JIS A 2103         | 樹脂           | 2.2                                    |
| 1    |                    | アルミ樹脂複合 | 4.7                                    |
| 2    |                    | アルミニウム   | 6.6                                    |

$ F_{de,ex} $の代表的な値を以下に示す（参考：平成25年省エネルギー基準に準拠した算定・判断の方法及び解説 II 住宅 第四部第三節及びJIS A 2103：2014 表A.2）。  

| ID   | 規格・基準         | 開閉形式 | 屋外側展開面積比$F_{de,ex}$ |
| ---: | :---               | :---     | ---:                        |
| 0    | 省エネ基準(住宅)   | ―       | 1.0                         |
| 1    | 省エネ基準(非住宅) | ―       | 1.0                         |
| 2    | JIS A 2103         | FIX      | 1.9                         |
| 3    |                    | 開き     | 1.5                         |
| 4    |                    | 引違い   | 2.1                         |
| 5    |                    | 上げ下げ | 1.6                         |

$ h_{fr,ex} $の代表的な物性値を以下に示す（参考：平成25年省エネルギー基準に準拠した算定・判断の方法及び解説 II 住宅 第四部第三節及びJIS A 2103：2014 5.3）。  

| ID   | 規格・基準         | 季節 | 屋外側表面熱伝達率$h_{fr,ex}$ [W/(m<sup>2</sup>･K)] |
| ---: | :---               | :--- | ---:                                                |
| 0    | 省エネ基準(住宅)   | ―   | 23                                                  |
| 1    | 省エネ基準(非住宅) | ―   | 25 (=1/0.04)                                        |
| 2    | JIS A 2103         | 夏期 | 14                                                  |
| 3    |                    | 冬期 | 24                                                  |

In [3]:
def hourly_gvalue_frame(L):
    if L.im == 0:     #表参照入力
        ghf_a = frame_prop(L)[0]
        ghf_U = frame_prop(L)[1]
        ghf_F = frame_prop(L)[2]
        ghf_h = frame_prop(L)[3]
    elif L.im == 1:   #直接入力
        ghf_a = L.ap
        ghf_U = L.Up
        ghf_F = L.Fp
        ghf_h = L.hp
    
    return ghf_a * ghf_U / (ghf_F * ghf_h)

def frame_prop(L):
    r_ghf_a = [0.8, 0.8, 0.69, 0.68, 0.67, 0.22, 0.87, 0.76, 0.47, 0.26, 0.24]
    r_ghf_U = [2.2, 4.7, 6.6]
    r_ghf_F = [1., 1., 1.9, 1.5, 2.1, 1.6]
    r_ghf_h = [23., 25., 14., 24.]
    
    return r_ghf_a[L.ap], r_ghf_U[L.Up], r_ghf_F[L.Fp], r_ghf_h[L.hp]

#### Example

In [4]:
if __name__ == '__main__':

    ghf_in = ghf_input(0, 0, 0, 0, 0)

    ghf = hourly_gvalue_frame(ghf_in)

    print(ghf)

0.0765217391304348
